In [ ]:
! pip install langchain langchain-community neo4j

Vector Search

In [ ]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings.ollama import OllamaEmbeddings

#make sure you have env variable set NEO4J_PLUGINS=["apoc"]
url = "bolt://localhost:7687"
username="neo4j"
password="password"

loader = TextLoader("/home/jonot480/Documents/paulgraham.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

embeddings = OllamaEmbeddings(model="nomic-embed-text")

# The Neo4jVector Module will connect to Neo4j and create a vector index if needed.

db = Neo4jVector.from_documents(
    docs, embeddings, url=url, username=username, password=password
)

query = "What is Charlie good at?"
docs_with_score = db.similarity_search_with_score(query, k=2)

Metadata Filtering

In [ ]:
db = Neo4jVector.from_documents(
    docs,
    embeddings,
    url=url, username=username, password=password
)

query = "What is Charlie good at?"
filter = {"name": {"$eq": "charlie"}}

docs = db.similarity_search(query, filter=filter)
print(docs)

Neo4j Graph

In [ ]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(url=url, username=username, password=password)

QUERY = """
MATCH (bacon:Person {name:"Kevin Bacon"})-[*1..4]-(hollywood)
RETURN DISTINCT hollywood
"""

graph.query(QUERY)